In [ ]:
import math
import random
import numpy as np
from graphviz import Digraph
from collections import deque
import matplotlib.pyplot as plt
# Ensure inline plotting
%matplotlib inline

In [ ]:
def trace(root):
    # Build a set of all nodes and edges in the graph
    nodes, edges = set(), set()
    
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    
    build(root)
    return nodes, edges

def draw_dot(root):
    dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'})  # Left to Right layout

    nodes, edges = trace(root)
    for n in nodes:
        uid = str(id(n))
        # For each value in the graph, create a rectangular ('record') node for it
        dot.node(name=uid, label="{%s | data: %.4f | grad: %.4f}" % (n.label, n.data, n.grad), shape='record')
        
        if n._op:
            # If the value has an operation, create a node for it
            dot.node(name=uid + n._op, label=n._op)
            # Create an edge from the operation node to the value node
            dot.edge(uid + n._op, uid)

    for n1, n2 in edges:
        # Create an edge from n1 to the operation node of n2
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)
    
    return dot

In [ ]:
class Value:
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label = label

    def __repr__(self):
        return f"Value(data={self.data})"

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        return out

    def __radd__(self, other):  # For commutative property of addition
        return self + other

    def __mul__(self, other):   # For multiplication
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out

    def __rmul__(self, other):  # For commutative property of multiplication
        return self * other

    def __pow__(self, other):   # For exponentiation
        assert isinstance(other, (int, float)), "only supporting int/float for now"
        if self.data == 0 and other < 0:
            raise ZeroDivisionError("0 cannot be raised to a negative power")
        out = Value(self.data**other, (self,), f'**{other}')

        def _backward():
            self.grad += other * self.data**(other-1) * out.grad
        out._backward = _backward
        return out

    def __truediv__(self, other):
        return self * other**-1

    def __rtruediv__(self, other):
        return Value(other) * self**-1

    def __neg__(self):
        return self * -1

    def __sub__(self, other):
        return self + (-other)

    def __rsub__(self, other):
        return Value(other) + (-self)

    def tanh(self):    # Activation function
        x = self.data
        e2x = math.exp(2*x)
        t = (e2x - 1) / (e2x + 1)
        out = Value(t, (self,), 'tanh')

        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        return out

    def exp(self):   # Exponential function
        x = self.data
        out = Value(math.exp(x), (self,), 'exp')

        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward
        return out

    # Implementing the backward pass
    def backward(self):
        topo, visited = deque(), set()

        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.appendleft(v)
        build_topo(self)

        self.grad = 1.0
        for node in topo:
            node._backward()

In [ ]:
class Neuron:
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1.0, 1.0)) for _ in range(nin)]
        self.b = Value(random.uniform(-1.0, 1.0))

    def __call__(self, x):
        act = sum((xi * wi for xi, wi in zip(x, self.w)), self.b)  # Weighted sum
        return act.tanh()  # Activation function
    
    def parameters(self):
        return self.w + [self.b]  # Ensuring it's not a mutable reference

class Layer:
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]

    def __call__(self, x):
        out = [n(x) for n in self.neurons]
        return out[0] if len(out) == 1 else out
    
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:
    def __init__(self, nin, nouts):
        sizes = [nin] + nouts
        self.layers = [Layer(sizes[i], sizes[i+1]) for i in range(len(sizes)-1)]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)  # Passing through each layer
        return x
    
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]


In [ ]:
# Define input and expected outputs
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0]

# Initialize the model
n = MLP(3, [4, 4, 1])  # MLP with 3 inputs, 2 hidden layers, and 1 output

In [ ]:
# Input and output before training
for x in xs:
    print(f"Before training: Input: {x}, Output: {n(x).data}")

In [ ]:
# Training loop
losses = []
for k in range(50):
    # Forward pass
    ypred = [n(x) for x in xs]
    loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))  # MSE loss

    # Zero gradients
    for p in n.parameters():
        p.grad = 0.0
    loss.backward()     # Backward pass
    
    # Update parameters with gradient descent
    for p in n.parameters():
        p.data += -0.2 * p.grad  # Learning rate: 0.2

    # Store loss for visualization
    losses.append(loss.data)
    print(f"Epoch {k}: Loss = {loss.data}")


In [ ]:
ypred

In [ ]:
# Visualization of loss over time
plt.plot(losses)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training Loss Over Time")
plt.show()

In [ ]:
# Visualizing the computation graph
dot = draw_dot(loss)  # Visualize loss computation graph
dot.render(view=True)  # Open the visualization